# Part I - Prosper Loan Data Analysis
## by Johnny Samuel

## Introduction
> This data set contains 113,937 loans with 81 variables on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, and many others.





## Preliminary Wrangling


#### We Import all the needed libraries that wil be useful during our anaysis.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the dataset
df=pd.read_csv('prosperLoanData.csv',encoding='ISO-8859-1')

In [ ]:
df.head()

In [ ]:
# Find the shape of the data set
df.shape

In [ ]:
# Give list of the column to enable us see all the columns and thus find the right columns to work with
col_list=list(df.columns)
col_list

In [ ]:
#### We try to observe the different columns

In [ ]:
df.PercentFunded.describe()

In [ ]:
df.Term.value_counts()

In [ ]:
df.EmploymentStatusDuration.value_counts()

In [ ]:
df.LoanOriginalAmount.value_counts().hist(bins=20)

In [ ]:
df.InvestmentFromFriendsCount.value_counts()

In [ ]:
# List of the columns we want to consider and draw inference from
loan_col=['LoanOriginalAmount', 'BorrowerAPR', 'StatedMonthlyIncome', 'Term', 'ProsperRating (Alpha)', 
        'EmploymentStatus','Occupation','BorrowerRate','LoanStatus']

In [ ]:
# we make a new dataframe which is the subset of the dataset
loan=df[loan_col]

In [ ]:
# we use the decribe function to see key statistical values for all numerical variables
loan.describe()

In [ ]:
# We observe the first 5 rows of the new dataset
loan.head()

### What is the structure of your dataset?

> This dataset conatins (113937 rows and 81 columns. The columns contain the vaariables that could contibute to the Loan Status

### What is/are the main feature(s) of interest in your dataset?

> The interst we want to focus on is understanding what affects the Loan Original Amount that the Borrowers Demand. We also hope to gain insight into facteors that affect the Borrowers annual Percentage Rate
 
        

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> The Features we will be using to support this inference include:
* Loan Original Amount
* BorrowerAPR
* Stated Monthly Income
* Term
* Prosper Rating (Alpha)', 
* Employment Status
* Occupation
* BorrowerRate
* LoanStatus

> #### Note: for the sake of limited time we will not draw insight from all variables though they will all have some insghts that could be observed.

## Univariate Exploration


### Insight on Employment status

In [ ]:
def employ():
    plt.figure(figsize=(8,6))
    color=sns.color_palette()[1]
    sns.countplot(data=loan, x='EmploymentStatus', color=color)
    plt.xticks(rotation=90)
    plt.title('Count of Employment Status')
    
employ()

#### Observation
> We observe that those that are *Employed* have the highest count, which is followed by the Full-time. This could be as a result of the salary they have which could serve as colateral

In [ ]:
loan.Occupation.value_counts()

### Insights from the Loan Original Amount

In [ ]:
def loanamount():
    plt.figure(figsize=(10,8))
    color=sns.color_palette()[1]
    plt.hist(data=loan, x='LoanOriginalAmount', color=color)
    plt.xticks(rotation=90)
    plt.title('Count of Employment Status')
    
loanamount()

#### Observation
> This was a rough analysis of the the Original amount that was loaned to each Borrower, but is better interpreted below

In [ ]:
log_binsize = 0.05
bins_log = 10 ** np.arange(3, np.log10(loan['LoanOriginalAmount'].max())+log_binsize, log_binsize)
plt.hist(data = loan, x = 'LoanOriginalAmount', bins = bins_log)
plt.xscale('log')
x_ticks = [1000, 2000, 3000, 4000, 5000, 10000, 15000,20000, 30000]
x_ticks_names = ['1k','2k', '3k', '4k', '5k', '10k','15k', '20k','30k']
plt.xticks(x_ticks, x_ticks_names)
plt.xlim((1000,35000))
plt.xlabel('Loan Original Amount ($)')
plt.title('Distribution of Loan Original Amount ($)');

#### Observation

> we discover that most of the Borrowers Collected money in multiples of five(5). This is may be due to the normal human idea mentality for easy remembring of the amount borrowed,

### Insights from the Time Frame(Month) to pay the total money borrrowed 

In [ ]:
def term():
    #plt.figure(figsize=(8,7))
    color=sns.color_palette()[1]
    sns.countplot(data=loan, x='Term', color=color)
    plt.xticks(rotation=90)
    plt.title('Count of Length of Term(Month)')
    
term()

#### Observation 

> Its observed that majority of the borrrowers settled to pay the money within 3 years, this may be because of the Employment status of each borrower it could have been easier to pay within 3 years than one year but making it five(5) year will also include extra cost for intrest.

### Insights from the Prosper Rating (Alpha)

In [ ]:
def prosper():
    plt.figure(figsize=(10,8))
    base_color=sns.color_palette()[1]
    sns.countplot(data=loan, x='ProsperRating (Alpha)', color=base_color)
    plt.xticks(rotation=90)
    plt.title('Count of Employment Status')
    
prosper()

#### Observation

> Notes: It is observed that Prosper Rating of C has the maximum number of borrowers thid is followed by B, A and D in that order

### Insights into Occupation

In [ ]:
# creating the text variable
text1 = loan.loc[:, 'Occupation'].str.cat(others=None, sep=' ')
# Creating word_cloud with text as argument in .generate() method

word_cloud1 = WordCloud(collocations = False, background_color = 'white',
                        width = 2048, height = 1080).generate(text1)
# saving the image
word_cloud1.to_file('map.png')

# Display the generated Word Cloud
plt.figure(figsize=(12,8))
plt.imshow(word_cloud1, interpolation='bilinear')
plt.axis("off")
plt.title('Statistics of Top Occupation')
plt.show()

#### Onservations
> Because of the differnt occupation by the borrowers it will be easier to use word cloud to get the occupation of most borrowers, they seem to be Professsionals, Computer Programmers and Executives

### Insights into Stated Monthly Income

In [ ]:
# cateegorise the stated monthly income to better understand the data 
def classify(row):
    if row["StatedMonthlyIncome"]<1000:
        return 'Low'
    elif row["StatedMonthlyIncome"]<4000:
        return 'Lower-Middle'
    elif row["StatedMonthlyIncome"]<12000:
        return 'Upper-Middle'
    else:
        return 'High'
    
loan['WageGroup'] = loan.apply(classify, axis=1)

wage_cat = ['Low', 'Lower-Middle','Upper-Middle', 'High']
order_wage = pd.api.types.CategoricalDtype(ordered = True, categories = wage_cat)
loan['WageGroup'] = loan['WageGroup'].astype(order_wage)

color=sns.color_palette()[1]
sns.countplot(data = loan, x = 'WageGroup', color = color);
plt.title('Borrowers Monthly Income($) Category ')
plt.xlabel('Income Category')
plt.ylabel('Amount($)')

#### Observation
>Notes: we Observe after categorising the income amount that majority are either upper-middle or Lower-Middle Income earners.

### Insight into Borrowwer APR 

In [ ]:
bins = np.arange(0, loan['BorrowerAPR'].max()+0.01, 0.01)
plt.hist(data = loan, x = 'BorrowerAPR', bins=bins);
plt.title('Count of Borrowers APR')
plt.xlabel('Borrower APR')
plt.ylabel('count')

#### Observation
> Notes: We observe that the histogram is skewed to the right with an outlier at 0.35 which may be error

### Insight into Loan Status

In [ ]:
# we categorise all past due dates together since all is past 
loan['LoanStatus'] = loan['LoanStatus'].apply(lambda x: x if 'Past Due' not in x else 'Past Due')

status_list = ['Defaulted','Chargedoff', 'Past Due', 'Current', 'FinalPaymentInProgress', 'Completed']

loan_status_ordering = pd.api.types.CategoricalDtype(ordered = True, categories = status_list)

loan['LoanStatus'] = loan['LoanStatus'].astype(loan_status_ordering)


status_count=loan['LoanStatus'].value_counts()
status_order=status_count.index

base_color = sns.color_palette()[1]
sns.countplot(data=loan, x='LoanStatus', color=base_color, order=status_order);
plt.title('Count of borrowers loan status ')

# get the current tick locations and labels
locs, labels = plt.xticks(rotation=90) 


#### Observation
> Notes: It is observed that majority of them are Currently still owe and a good percentage also have completed their payment. This is good since with that will prevent bankrupcy

## Bivariate Analysis

### Insight into the Term and original amount loan to them

In [ ]:
# plot boxplot
plt.figure(figsize = [8,5])
sns.boxplot(data = loan, x = 'Term', y = 'LoanOriginalAmount' ,color = color);
plt.title('Term Vs Loan Original Amount')
plt.ylabel('Loan Amount($)')
plt.xlabel('Term')

#### Observation
> Notes: The higher the term the higher the amount, it is also seen that the median of the three years duration is same as the maximum amount for the duration of one year, also there are outliers that make the maximum for each of the term

### Insights of Borrower Rate with Employment Status, Prosper Rating and Wage Group

In [ ]:
#define boxgrid function
def boxgrid(x, y, **kwargs):
    base_color = sns.color_palette()[1]
    sns.boxplot(x, y, color = base_color)
    plt.xticks(rotation=90);


g = sns.PairGrid(data = loan, y_vars = ['BorrowerRate'], x_vars = ['EmploymentStatus','ProsperRating (Alpha)','WageGroup'],
                size = 3, aspect = 1.5);
g.map(boxgrid);
plt.suptitle(' Borrower APR Vs Employment Status,Prosper Rating(Alpha), Wage Group')

#### Observation
> Notes:
* Those having some source of employment seems to have the same range in Borrowers Rate whille those without employment have a higher rate
* Prosper Rating has as the Prosper rating increase there is a corresonding reduction in the Borrower rate
* A decrease in the Wage group income leads to a corresponding decrease in the Borrower rate

### Insights into the Loan original amount from Employment,Prosper Rating and Wage Group

In [ ]:
#define boxgrid function
def boxgrid(x, y, **kwargs):
    base_color = sns.color_palette()[1]
    sns.boxplot(x, y, color = base_color)
    plt.xticks(rotation=90);


g = sns.PairGrid(data = loan, y_vars = ['LoanOriginalAmount'], x_vars = ['EmploymentStatus','ProsperRating (Alpha)','WageGroup'],
                size = 3, aspect = 1.5);
g.map(boxgrid);
plt.ylabel('Loan Amount')
plt.suptitle('Loan Amount Vs Employment Status,Prosper Rating(Alpha), Wage Group')

#### Observation
> Notes:
* Those employed seem to collect more loan than other this is no suprise because of the steady source of income it give them the assurance of ability to pay, unlike the retiree and not-employed who also collect just small amount this will be to settle little matters.
* It is observed that those with high prosper rating also borrowed bigger amount
* It is worthy of note that the greater the borrowers Wage/Salary the greater the amount borrowed.

### Insight into the correlation between each quantities

In [ ]:
sns.heatmap(loan.corr(), annot = True, fmt = '.3f',
           cmap = 'YlGnBu', center=0)
plt.title('Correlation Between each quantities')
plt.show()

#### Observations
>Notes: with regards to Loan original amount; The Term has a postive correlation and further study on its contribution will be important. While with regards to BorrowerAPR there is a strong positive correlation of about 0.99

### Insight into The Loan Status with Term, and wage group

In [ ]:
plt.figure(figsize = [10, 16])

# subplot 1: Loan Status vs term
plt.subplot(3, 1, 1)
sns.countplot(data = loan, x = 'LoanStatus', hue = 'Term')
plt.xticks(rotation=10)
plt.xlabel('Loan Status')
plt.title('Loan Status vs Term')
# subplot 2: Employment Status vs Wage Group
ax = plt.subplot(3, 1, 2)
sns.countplot(data = loan, x = 'WageGroup', hue = 'Term')
ax.legend(loc = 1, ncol = 2);
# re-arrange legend to remove overlapping
plt.xticks(rotation=10);
plt.xlabel('Employment Status')
#plt.title('Employment status vs Wage Group')

#### Observtion
>Notes:
   
   * We discovered that those with one year term have completed theirs and also five(5) year term is more for the current loan status
   * Also those of low income doesn't have much of the one year term this may be becuase of the monthly amount that will be collected. also unlike expected the high income earners didnt borrow over a five(5) year term.

### Insight into the Borrower APR and original amount

In [ ]:
plt.figure(figsize = [8, 6])
sns.regplot(data = loan, x = 'LoanOriginalAmount', y = 'BorrowerAPR', scatter_kws={'alpha':0.01});
plt.title('Correlation Between BorrowerAPR and Loan Original Amount')
plt.xlabel('Original Loan amount($)')
plt.ylabel('Borrower APR')

#### Observation

> Notes:
    It is observed that it is negatively correlated that is a decrease in the loan amount leads to an increase in the BorrowerAPR

## Multivariate

### Insight into the Wage Group Loan Amount and Term

In [ ]:
# we plot a line graph of wage group and loan amount against the Term
plt.figure(figsize=(10,10))
g=sns.lineplot(data=loan, x='WageGroup', y='LoanOriginalAmount', hue='Term',ci=None);
plt.title('Wage Group Vs Loan Original Amount Vs Term')
plt.ylabel('Loan Amount($)')
plt.xlabel('Wage Group')

#### Observation


> Notes: Its observed that those with higher term also borrow higher amount and also the higher the borrowers income the higher the term and the higher the amount borrowed, this should be because of the collateral they have.

### Insights into the Wage Group,Borrowerr APR and Term 

In [ ]:
#plot boxplot
plt.figure(figsize = [8,5])
sns.boxplot(data = loan, x = 'WageGroup', y = 'BorrowerAPR',hue = 'Term' ,color = color);
plt.title('Wage Group Vs BorrowerAPR Vs Term')

#### Observation

>Notes: we observed that the borrower APR was higher for a year than for other term. Also the three year term has the highest borrower APR

### Insight into the loan status,Stated monthly income and the loan original amount

In [ ]:
sedan_classes = ['Defaulted', 'Chargedoff', 'Past Due', 'Current', 'FinalPaymentInProgress','Completed']
pd_ver = pd.__version__.split(".")
if (int(pd_ver[0]) > 0) or (int(pd_ver[1]) >= 21): # v0.21 or later
    LoanStatuses = pd.api.types.CategoricalDtype(ordered = True, categories = sedan_classes)
    loan['LoanStatus'] = loan['LoanStatus'].astype(LoanStatuses)
else: # compatibility for v.20
    loan['LoanStatus'] = loan['LoanStatus'].astype('category', ordered = True, categories = sedan_classes)
  # plotting
g = sns.FacetGrid(data = loan, col = 'LoanStatus', size = 4, col_wrap = 3)
g.map(plt.scatter, 'StatedMonthlyIncome','LoanOriginalAmount', alpha = 1/5)


#### Observation
> Notes:we discovered that those who are currently paying the money borrowed borrow the highest mount, also those that are defaulters seems to owe for long hence the amount borrowed. Also those the arc are thinner for tose in final payment in progress this is because not many persons are in this category.

### Reference
* https://blogs.worldbank.org/opendata/new-world-bank-country-classifications-income-level-2020-2021